# RoboSINDy: Project Introduction

**Authors**: Julian Skifstad and Advaith Balaji

This notebook implements the paper "Data-driven discovery of coordinates and governing equations" by Champion et al. on the Franka Panda Arm. The goal is to compare this method with e2c. This work aims to explore and evaluate the performance of these approaches in a robotic planar pushing task.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from numpngw import write_apng
from IPython.display import Image
from tqdm.notebook import tqdm
import time

In [ ]:
from utils.panda_pushing_env import PandaImageSpacePushingEnv
from utils.visualizers import GIFVisualizer, NotebookVisualizer
from utils.utils import *
from sindy.SINDy import RoboSINDy, SindyDataset, NormalizationTransform

Visualize panda arm environment in gym. Executes 3 random pushes.

In [ ]:
%matplotlib inline

# Create the visualizer
fig = plt.figure(figsize=(8,8))
hfig = display(fig, display_id=True)
visualizer = NotebookVisualizer(fig=fig, hfig=hfig)

# Initialize the simulation environment
env = PandaImageSpacePushingEnv(visualizer=visualizer,
                                render_non_push_motions=True,
                                camera_heigh=800,
                                camera_width=800,
                                grayscale=True,
                                done_at_goal=False)
env.reset()
# Perform a sequence of 3 random actions:
states = []
for i in tqdm(range(3)):
    action_i = env.action_space.sample()
    state, reward, done, info = env.step(action_i)
    states.append(state)
    if done:
        break

view_states(states)

plt.close(fig)

Collect a dataset npy file that also included the state_derivatives. This will open the npy file in dataset_path and compute the state discrete time derivatives using central difference formula. It will then repackage states, actions, and state derivatives into a new .npy file which we can use as our dataset.

In [ ]:
dataset_path = "datasets/pushing_image_data.npy"
dt = 1/240.0


data_npy = np.load(dataset_path, allow_pickle=True)

samples = []
for item in data_npy:
    
    states = item['states']
    actions = item['actions']
    state_derivatives = []
    for i in range(1, len(states)-1):
        state_derivative = (states[i+1] - states[i-1]) / dt
        state_derivatives.append(state_derivative)
    state_derivatives = np.array(state_derivatives)

    for i, state in enumerate(states[1:-1]):
        sample = {
            'state': state,
            'action': actions[i],
            'state_derivative': state_derivatives[i]
        }
        samples.append(sample)

# print info about samples
print(f"Number of samples: {len(samples)}")
print(f"Sample state shape: {samples[0]['state'].shape}")
print(f"Sample action shape: {samples[0]['action'].shape}")
print(f"Sample state derivative shape: {samples[0]['state_derivative'].shape}")

In [ ]:
# Some training and datset parameters
val_fraction = 0.2
batch_size = 64
num_epochs = 75000

The cell below will construct the Dataset used for RoboSINDy. It reads each sample from the .npy samples we created, normalizes each state and state derivatives, and constructs a dataloader to be used for training.

In [ ]:
dataset = SindyDataset(data=samples)

val_size = int(val_fraction * len(dataset))
train_size = len(dataset) - val_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)

tot_train_states = []
tot_train_state_derivatives = []
for i in range(len(train_loader.dataset)):
    s = train_loader.dataset[i]['states']   
    sd = train_loader.dataset[i]['state_derivatives']  
    tot_train_states.append(s.unsqueeze(0))
    tot_train_state_derivatives.append(sd.unsqueeze(0))
tot_train_states = torch.cat(tot_train_states,dim=0)  
tot_train_state_derivatives = torch.cat(tot_train_state_derivatives, dim=0)  # (N,C,H,W)

mean_s = tot_train_states .mean(dim=(0,2,3))
std_s = tot_train_states .std( dim=(0,2,3))
mean_sd = tot_train_state_derivatives.mean(dim=(0,2,3))
std_sd = tot_train_state_derivatives.std( dim=(0,2,3))

normalization_constants = {
  'mean_state': mean_s,              
  'std_state':  std_s,               
  'mean_state_derivative': mean_sd,  
  'std_state_derivative': std_sd,    
}

norm_tr = NormalizationTransform(normalization_constants)
train_data.dataset.transform = norm_tr
val_data.dataset.transform = norm_tr

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)

print(f"Loaded {len(train_loader.dataset)} training samples and {len(val_loader.dataset)} validation samples.\n")

# get one sample and check quality
sample = train_loader.dataset[0]
print(f"Sample state shape: {sample['states'].shape}")
print(f"Sample action shape: {sample['actions'].shape}")
print(f"Sample state derivative shape: {sample['state_derivatives'].shape}")
print(f"Sample state: {sample['states']}")


In [ ]:
#define the model
model = RoboSINDy(input_dim=32*32*1, batch_size=batch_size)

model.train_model(train_loader, num_epochs=num_epochs, learning_rate=0.001)

#save the model
model_path = 'trained_models/sindy_model.pt'
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
print("Xi coefficients:")

# print in decimal form
for i in range(len(model.xi_coefficients)):
    for j in range(len(model.xi_coefficients[i])):
       print(model.xi_coefficients[i, j].detach().numpy(), end=", ")
    print("\n")